In [1]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
X = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_train.ftr')
y = pd.read_feather('../input/amex-imputed-and-1hot-encoded/y_train.ftr')

X = X.set_index('customer_ID')
y = y.set_index('customer_ID')['target']

Function to calculate competition's evaluation metric: https://www.kaggle.com/code/inversion/amex-competition-metric-python

In [3]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [4]:
rfc = RandomForestClassifier(max_depth=5)
rfc = rfc.fit(X, y)

In [5]:
proba_preds = rfc.predict_proba(X)[:, 1]
preds = rfc.predict(X)

In [6]:
print(f'M = {amex_metric(pd.DataFrame(y), pd.DataFrame(proba_preds, index=y.index, columns=["prediction"]))}')

M = 0.6340978403530687


In [7]:
print(classification_report(y, preds))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91   4153582
           1       0.74      0.65      0.69   1377869

    accuracy                           0.86   5531451
   macro avg       0.81      0.79      0.80   5531451
weighted avg       0.85      0.86      0.85   5531451



In [8]:
pickle.dump(rfc, open('random_forest_classifier_model.sav', 'wb'))